In [100]:
# assumptions
### players move at random
### starting position choosen at random
### players can move on the same position
### near means in all 8 directions (adjacent iff distance <= sqrt(2))
### all player must move (no one is stucked)
### movement speed is costant and the same for all the players
### discrete time
### discrete space (map like chessboard)
### each player shoots at the same time

# input parameters
### size of the area (m^2)
### mobility speed (m/s)
###         if speed is 1 m/s each player moves 1 slot at each second
###         if speed is 10m/s each player moves 1 slot each 0.1 second
#
### number of initial players

# output metrics
### time to win (s)
### killed opponents

# main data structures
### NO DATA STRUCTURE FOR FES (compute all movement and kills at each for loop)
### player: id
###         position (x, y)
###         status (killed or not)
###         killed opponents
###
### list of players

# main algorithms
### movement algorithm: at random add or subtract either 1 or 0 for both x and y coordinates
###                     coordinates can't be larger the size of the board
###
### find if is near algorithm: compute distance between 2 points, if less than sqrt(2) they are near 
###
### shoot algorithm: for each player computes near player
###                  if you find one, choose one of the 2 at random and set status to killed
###
### simulation: loop while there are players alive, at each iteration compute movement, shooting

# possible extension
### add life (resitance to more than one shoot)
### add strenght of the shoot (also possible to compute it in function of the distance between players)
### ability of the player or in the same way the stats of the weapon/player
### reducing the size of the map
### obstacoles in the map (forbidden zones, different movement speed in some areas)

In [ ]:
import numpy as np
import pandas as pd
import math
import random
import statistics

In [ ]:
seed = 53
np.random.seed(seed)
random.seed(seed)

area = 7
inital_players = 4
remaining_players = inital_players
movement_speed = 1
epoch = 0  # used to compute the time to win
# max_turns = 20

print("PARAMETERS\n")
print("area: {}x{} m^2".format(area, area))
print("initial players: {}".format(inital_players))
print("movement speed: {} m/s".format(movement_speed))

In [ ]:
# return a random position in the map
def startingPosition():
    x = np.random.randint(0, area)
    y = np.random.randint(0, area)
    return (x, y)

In [ ]:
# move +-1 in x and y dimension
def move(x, y):
    x_movement = np.random.randint(-1, 2)
    y_movement = np.random.randint(-1, 2)
    # print(x_movement, y_movement)
    new_x = x + x_movement
    new_y = y + y_movement

    if (new_x < 0 or new_x >= area):  # new x position is not valid
        new_x = x
    if (new_y < 0 or new_y >= area):  # new y position is not valid
        new_y = y

    return (new_x, new_y)

In [ ]:
# find if two players are near
def isNear(coord1, coord2):
    distance = math.dist(coord1, coord2)
    if distance < 1.42:  # distance less than sqrt(2) the players are near 
        return True
    else:
        return False

In [ ]:
def shooting(players):
    pass

In [ ]:
class Player:
    def __init__(self, id, x, y):
        self.position = (x, y)
        self.id = id
        self.killed = False  
        self.killedOpponents = 0
        self.winner = False

    def setPosition(self, position):
        self.position = position
    
    def setKilled(self):
        self.killed = True  
        
    def increasKilledOpponents(self):
        self.killedOpponents += 1

    def setIsWinner(self):
        self.winner = True

    def getId(self):
        return self.id

    def getPosition(self):
        return self.position

    def getKilledOpponents(self):
        return self.killedOpponents

    def isKilled(self):
        return self.killed

    def isWinner(self):
        return self.winner


In [ ]:
players = []  # list of players
# populate players
for id in range(0, inital_players):
    x, y = startingPosition()
    p = Player(id, x, y)
    players.append(p)
    print(f'player [{id}] added to the game')
    print(f'\tstarting position for player [{id}] is ({x}, {y})')

print()

In [ ]:
# while (remaining_players > 1 and epoch < max_turns):
while (remaining_players > 1):
    print(f'turn [{epoch}]')
    # if epoch % 1000 == 0:
        # print(f'turn [{epoch}]')

    # move each player 
    for p in players:
        if p.isKilled() is True:  # skip killed player
            continue

        current_position = p.getPosition()
        # compute new position
        new_position = move(current_position[0], current_position[1])
        p.setPosition(new_position)

        print(f'player [{p.getId()}] moved in ({new_position[0]}, {new_position[1]})')

    # shooting
    for p in players:
        if p.isKilled() is True:  # skip killed player
            continue

        for _p in players:
            if p.getId() == _p.getId():  # same player
                continue
            if _p.isKilled() is True:
                continue

            if isNear(p.getPosition(), _p.getPosition()):  # if players are near
                if (np.random.randint(0, 2) == 0):  # p wins
                    p.increasKilledOpponents()
                    _p.setKilled()
                    remaining_players += -1
                    print(f'player [{p.getId()}] killed player [{_p.getId()}]')
                else:  # _p wins
                    _p.increasKilledOpponents()
                    p.setKilled()
                    remaining_players += -1
                    print(f'player [{_p.getId()}] killed player [{p.getId()}]')

        
    random.shuffle(players)  # shuffle list to change the order used to search for near players
    epoch += 1

print(f"REMAINING PLAYERS = {remaining_players}")

if remaining_players == 1:
    for p in players:
        if p.isKilled() is False:
            p.setIsWinner()
            print(f'PLAYER [{p.getId()}] WINS!!')

if remaining_players == 0:
    print("ERROR -> remaining_player = 0")

In [ ]:
# print statistics and compute average of killed opponents for each players
killed_opponents = []
winner_killed_opponents = -1
for p in players:
    killed_opponents.append(p.getKilledOpponents())
    if p.isWinner() is True:
        winner_killed_opponents = p.getKilledOpponents()  # save killed opponents of the winner

    print('PLAYER {} INFO:'.format(p.getId()))
    print('\tis winner: {}'.format(p.isWinner()))
    print('\tis killed: {}'.format(p.isKilled()))
    print('\t# killed opponents: {}'.format(p.getKilledOpponents()))
    print('\tlast position: {}'.format(p.getPosition()))
    print()

In [ ]:
remaining_players

In [ ]:
df = pd.DataFrame(columns=['time', 'initial_players', 'area', 'mobility_speed', 'seed', 'avg_killed_opponents', 'winner_killed_opponents'])

l = [epoch * movement_speed, inital_players, area, movement_speed, seed, statistics.fmean(killed_opponents), winner_killed_opponents]

In [ ]:
df.loc[len(df)] = l
df

In [112]:
df = pd.DataFrame(columns=['time', 'initial_players', 'area', 'mobility_speed', 'seed', 'avg_killed_opponents', 'winner_killed_opponents'])

l = [epoch * movement_speed, inital_players, area, movement_speed, seed, statistics.fmean(killed_opponents), winner_killed_opponents]

In [113]:
df.loc[len(df)] = l
df

,time,initial_players,area,mobility_speed,seed,avg_killed_opponents,winner_killed_opponents
0,9.0,4.0,7.0,1.0,53.0,0.75,2.0
